In [1]:
# Parameters
EMO_INDEX = 2
MODEL_INDEX = 7
N = 20


In [2]:
import os
import ast
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.dates as mdates
import statsmodels.api as sm # 用于统计检验
%matplotlib inline
plt.rcParams["font.sans-serif"]=["WenQuanYi Micro Hei"] #设置字体
plt.rcParams["axes.unicode_minus"]=False #该语句解决图像中的“-”负号的乱码问题

## 参数选择

In [3]:
# # Parameters
# MODEL_INDEX = 2
# EMO_INDEX = 0
# N = 20

In [4]:
ORI_DATA_PATH = '/data/public/fintechlab/zdh/Individual-Stock-Analysis/B_Temporal_Clustering/data'
ROOT_PATH = '/data/public/fintechlab/zdh/Individual-Stock-Analysis/B_Result_Analysis'
CLUSTER_PATH = f'{ROOT_PATH}/data/Comparison_of_Emotional_models'

MODEL = ['Emo-Dict/DLUT', 'Emo-Dict/Bian', 'Emo-Dict/Jiang',
         'Machine-learning/LR', 'Machine-learning/RF', 'Machine-learning/SVM', 
         'Deep-learning/BERT', 'Deep-learning/Ours'][MODEL_INDEX]
Emotion_Data_PATH = f'{ORI_DATA_PATH}/Emotion_Data/{MODEL}'   # 情绪数据路径
Financial_Data_PATH = f'{ORI_DATA_PATH}/Financial_Data' # 金融数据路径

In [5]:
EMO_MARKET = ['Shanghai_Composite_Index', 'CSI_300_Index', 'Chinext_Index'][EMO_INDEX]
EMO_INDEX_MAP = {
    'Shanghai_Composite_Index': '上证综合情绪值',
    'CSI_300_Index': '沪深300情绪值',
    'Chinext_Index': '创业板情绪值'
}
EMO_NAME = EMO_INDEX_MAP[EMO_MARKET]

## 数据预处理

In [6]:
## 读取聚类结果
cluster_data = pd.read_csv(f'{CLUSTER_PATH}/{EMO_MARKET}/{MODEL}.csv', dtype={"Stkcd": str})
cluster_data.columns = ['股票编号', '聚类标签', '公司名称']

# 尝试抽样直到满足每个簇至少出现3次
for _ in range(100):
    cluster_data = cluster_data.sample(n=N, random_state=np.random.randint(0, 10000))
    label_counts = cluster_data['聚类标签'].value_counts()
    
    if all(label_counts >= 3):
        break
else:
    raise ValueError(f"在 {RETRY_LIMIT} 次尝试中无法找到满足每个簇至少3个样本的抽样结果，请减少 N 或检查数据分布。")

cluster_data.head()

,股票编号,聚类标签,公司名称
182,688981,2,中芯国际
116,600690,0,海尔智家
87,600166,2,福田汽车
122,600763,0,通策医疗
89,600198,0,大唐电信


In [7]:
## 读取股吧个股的数据
all_data = []
file_list = [f for f in os.listdir(Emotion_Data_PATH) if f.endswith('.csv')]

if MODEL != 'Deep-learning/Ours':
    for file in file_list:
        file_path = os.path.join(Emotion_Data_PATH, file)
        df = pd.read_csv(file_path)
        stock_code = os.path.splitext(file)[0] # 获取股票编号（文件名去掉扩展名）
        
        # 提取每一行的日期和情绪值
        for _, row in df.iterrows():
            new_row = {
                '股票编号': stock_code,
                '日期': row['日期'],
                '上证综合情绪值': row['上证综合情绪值'],
                '沪深300情绪值': row['沪深300情绪值'],
                '创业板情绪值': row['创业板情绪值']
            }
            all_data.append(new_row)
    guba_data = pd.DataFrame(all_data)
else:
    for file in tqdm(file_list, desc="读取文件"):
        file_path = os.path.join(Emotion_Data_PATH, file)
        try:
            df = pd.read_csv(file_path, usecols=['日期', '高维情绪变量'])  # 只读需要的列
        except Exception as e:
            print(f"读取失败 {file}: {e}")
            continue
    
        stock_code = os.path.splitext(file)[0]
    
        df['股票编号'] = stock_code
        all_data.append(df)
    guba_data = pd.concat(all_data, ignore_index=True)

guba_data

读取文件:   0%|          | 0/183 [00:00<?, ?it/s]

读取文件:   1%|          | 1/183 [00:00<01:04,  2.82it/s]

读取文件:   2%|▏         | 3/183 [00:00<00:38,  4.69it/s]

读取文件:   2%|▏         | 4/183 [00:01<01:18,  2.27it/s]

读取文件:   3%|▎         | 5/183 [00:01<00:59,  3.01it/s]

...

读取文件:  99%|█████████▉| 181/183 [01:02<00:00,  4.64it/s]

读取文件:  99%|█████████▉| 182/183 [01:03<00:00,  4.52it/s]

读取文件: 100%|██████████| 183/183 [01:04<00:00,  2.17it/s]

读取文件: 100%|██████████| 183/183 [01:04<00:00,  2.84it/s]

,日期,高维情绪变量,股票编号
0,2024-12-27,"[-0.18730907142162323, -0.22363345324993134, 0...",601933
1,2024-11-18,"[-0.6446929574012756, 0.14099551737308502, 0.4...",601933
2,2024-11-17,"[-0.14928916096687317, -0.10993610322475433, 0...",601933
3,2024-11-17,"[-0.2892271876335144, 0.09153766185045242, 0.2...",601933
4,2024-11-17,"[-0.18511946499347687, 0.3309328258037567, 0.2...",601933
...,...,...,...
495326,2021-04-09,"[-0.14503593742847443, 0.25753334164619446, 0....",601919
495327,2021-04-09,"[-0.30359137058258057, 0.26559382677078247, 0....",601919
495328,2021-04-08,"[0.15271537005901337, -0.08832913637161255, 0....",601919
495329,2021-04-08,"[-0.004539322108030319, 0.3122449517250061, 0....",601919


In [8]:
## 读取股票回报率的数据
return_data = pd.read_csv(f'{Financial_Data_PATH}/日个股回报率.csv', dtype={'股票编号': str})
return_data

,股票编号,日期,交易量,收益率变化
0,000002,2021-06-01,60990961,-0.003745
1,000002,2021-06-02,85354506,0.006015
2,000002,2021-06-03,50594187,-0.003363
3,000002,2021-06-04,71422364,-0.012748
4,000002,2021-06-07,64745280,-0.014812
...,...,...,...,...
154877,688981,2024-11-20,58507495,-0.017071
154878,688981,2024-11-21,56197106,0.002358
154879,688981,2024-11-22,79240108,-0.050588
154880,688981,2024-11-25,76905909,-0.029402


In [9]:
## 进行左连接，以 guba_data 为主表
merged_data = pd.merge(guba_data, return_data[['股票编号', '日期', '交易量', '收益率变化']], 
                       on=['股票编号', '日期'], 
                       how='left')
merged_data = pd.merge(merged_data, cluster_data, on='股票编号', how='left')
merged_data = merged_data.dropna()

merged_data['日期'] = pd.to_datetime(merged_data['日期'])
merged_data = merged_data[(merged_data['日期'] > '2021-05-06') & (merged_data['日期'] < '2024-11-18')]
merged_data

,日期,高维情绪变量,股票编号,交易量,收益率变化,聚类标签,公司名称
11194,2024-11-15,"[-0.21272552013397217, 0.11189557611942291, 0....",600685,15188430.0,-0.034014,1.0,中船防务
11195,2024-11-15,"[-0.05771368741989136, 0.05143539980053902, 0....",600685,15188430.0,-0.034014,1.0,中船防务
11196,2024-11-14,"[-0.29904043674468994, -0.10444869101047516, 0...",600685,19288530.0,-0.028634,1.0,中船防务
11197,2024-11-14,"[-0.5945582985877991, 0.06150205433368683, 0.1...",600685,19288530.0,-0.028634,1.0,中船防务
11198,2024-11-13,"[-0.23249807953834534, 0.10749123245477676, 0....",600685,28835039.0,0.035741,1.0,中船防务
...,...,...,...,...,...,...,...
481589,2021-06-03,"[-0.11042080074548721, 0.00951363891363144, 0....",002230,38880163.0,-0.025195,2.0,科大讯飞
481590,2021-06-03,"[-0.03181225433945656, 0.00998214352875948, 0....",002230,38880163.0,-0.025195,2.0,科大讯飞
481591,2021-06-03,"[-0.20233358442783356, 0.05385609343647957, 0....",002230,38880163.0,-0.025195,2.0,科大讯飞
481592,2021-06-03,"[-0.3143976032733917, 0.08426006883382797, 0.5...",002230,38880163.0,-0.025195,2.0,科大讯飞


In [10]:
# 对情绪值列进行 Min-Max 标准化
def min_max_normalization(df, cols):
    for col in cols:
        min_val = df[col].min()
        max_val = df[col].max()
        df[col] = 2 * (df[col] - min_val) / (max_val - min_val) - 1
    return df

# 对每个股票编号的数据进行标准化和按日期汇总
def process_data(df):
    if MODEL != 'Deep-learning/Ours':
        df_processed = pd.DataFrame()  # 用于存储结果
        for stock_code, stock_data in df.groupby('股票编号'):
            # 对每个股票编号内的数据进行标准化
            stock_data = min_max_normalization(stock_data, ['上证综合情绪值', '沪深300情绪值', '创业板情绪值'])
            
            # 按日期汇总数据，同时保留股票编号
            stock_summary = stock_data.groupby('日期').agg({
                '股票编号': 'first',  # 保留股票编号（在同一日期内它是相同的，使用 'first'）
                '上证综合情绪值': 'mean',  # 上证综合情绪值按日期取均值
                '沪深300情绪值': 'mean',  # 沪深300情绪值按日期取均值
                '创业板情绪值': 'mean',  # 创业板情绪值按日期取均值
                '交易量': 'mean',  # 交易量按日期求和
                '收益率变化': 'mean',  # 收益率变化按日期取均值
                '聚类标签': 'first', # 保留聚类标签（在同一日期内它是相同的，使用 'first'）
                '公司名称': 'first', # 保留公司名称（在同一日期内它是相同的，使用 'first'）
            }).reset_index(drop=False)
            
            # 合并每个股票的汇总数据
            df_processed = pd.concat([df_processed, stock_summary], ignore_index=True)
            df_processed = df_processed.sort_values(by=['股票编号', '日期'], ascending=[True, True])
        return df_processed
    else:
        df_processed = []    
        # 确保高维情绪变量是 list 类型（如果还没转过）
        if isinstance(df['高维情绪变量'].iloc[0], str):
            df['高维情绪变量'] = df['高维情绪变量'].progress_apply(ast.literal_eval)
        # 对每个股票编号处理，加 tqdm 进度条
        for stock_code, stock_data in tqdm(df.groupby('股票编号'), desc="Processing stocks"):
            for date, group in stock_data.groupby('日期'):
                emotion_matrix = np.array(group['高维情绪变量'].tolist())  # (样本数, 维度)

                avg_emotion = emotion_matrix.mean(axis=0).round(3).tolist()

                summary_row = {
                    '股票编号': stock_code,
                    '日期': date,
                    '高维情绪变量': avg_emotion,
                    '交易量': group['交易量'].mean(),
                    '收益率变化': group['收益率变化'].mean(),
                    '聚类标签': group['聚类标签'].iloc[0], # 保留聚类标签
                    '公司名称': group['公司名称'].iloc[0], # 保留公司名称
                }
                df_processed.append(summary_row)


    return pd.DataFrame(df_processed)

final_data = process_data(merged_data)
final_data

  0%|          | 0/37031 [00:00<?, ?it/s]

  0%|          | 14/37031 [00:00<04:24, 139.74it/s]

  0%|          | 73/37031 [00:00<01:31, 403.66it/s]

  0%|          | 132/37031 [00:00<01:16, 484.79it/s]

  1%|          | 191/37031 [00:00<01:10, 523.54it/s]

...

Processing stocks:  90%|█████████ | 18/20 [00:03<00:00,  5.65it/s]

Processing stocks:  95%|█████████▌| 19/20 [00:03<00:00,  5.96it/s]

Processing stocks: 100%|██████████| 20/20 [00:03<00:00,  5.19it/s]

Processing stocks: 100%|██████████| 20/20 [00:03<00:00,  5.48it/s]

,股票编号,日期,高维情绪变量,交易量,收益率变化,聚类标签,公司名称
0,000002,2021-06-01,"[-0.178, -0.141, 0.283, 0.184, -0.002, 0.299, ...",60990961.0,-0.003745,2.0,万科A
1,000002,2021-06-08,"[-0.276, -0.035, 0.247, 0.326, 0.018, 0.244, -...",44676494.0,0.004626,2.0,万科A
2,000002,2021-06-10,"[-0.083, 0.072, 0.273, 0.26, -0.087, 0.276, -0...",53800776.0,-0.010035,2.0,万科A
3,000002,2021-06-11,"[-0.114, 0.163, 0.381, 0.325, -0.091, 0.088, -...",75853738.0,-0.014035,2.0,万科A
4,000002,2021-06-15,"[-0.169, 0.147, 0.442, 0.245, -0.009, 0.368, -...",89915501.0,-0.020957,2.0,万科A
...,...,...,...,...,...,...,...
9679,688981,2024-11-11,"[-0.055, 0.026, 0.277, 0.147, -0.029, 0.276, -...",138069018.0,0.046660,2.0,中芯国际
9680,688981,2024-11-12,"[-0.134, -0.011, 0.308, 0.208, 0.049, 0.263, -...",108866759.0,-0.036864,2.0,中芯国际
9681,688981,2024-11-13,"[-0.228, 0.028, 0.358, 0.294, -0.137, 0.178, -...",80759477.0,-0.019484,2.0,中芯国际
9682,688981,2024-11-14,"[-0.18, 0.054, 0.392, 0.292, -0.048, 0.215, -0...",76194102.0,-0.022897,2.0,中芯国际


In [11]:
# 统计检验 : 按照聚类标签分组，并计算均值和方差
df = final_data.copy()
if MODEL != 'Deep-learning/Ours':
    statistical_result = df.groupby('聚类标签').agg(
        情绪值均值=(EMO_NAME, 'mean'),
        情绪值方差=(EMO_NAME, 'var'),
        收益率变化均值=('收益率变化', 'mean'),
        收益率变化方差=('收益率变化', 'var')
    ).reset_index()
else:
    statistical_result = None
# 输出结果
statistical_result

## 情绪值与股价二维展示图

In [12]:
## 绘制不同簇情绪值随时间变化图
if MODEL != 'Deep-learning/Ours':
    # 按 '聚类标签' 和 '日期' 分组，计算平均情绪值
    df_grouped = final_data.groupby(['聚类标签', '日期'])[['上证综合情绪值', '沪深300情绪值', '创业板情绪值']].mean().reset_index()
    
    # 为每个聚类标签绘制折线
    i = 0
    plt.figure(figsize=(12, 12))
    for cluster in df_grouped['聚类标签'].unique():
        if i == 0:
            x = 'red'
        elif i == 1:
            x = 'blue'
        elif i == 2:
            x = 'green'
        df_cluster = df_grouped[df_grouped['聚类标签'] == cluster]
        plt.plot(df_cluster['日期'], df_cluster[EMO_NAME], label=f'{EMO_NAME} - 聚类 {cluster}', color=x)
        i += 1
        
    plt.xlabel('日期')
    plt.ylabel('平均情绪值')
    plt.title('情绪值趋势图')
    
    
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator(bymonthday=15, interval=3))  # 每三个月显示
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  # 格式化为年-月
    
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    
    # plt.savefig(f'个股分析/{market_address}/图片/情绪值趋势图', dpi=300)
    plt.show()


In [13]:
## 绘制不同簇收益率随时间变化图
if MODEL != 'Deep-learning/Ours':
    # 按 '聚类标签' 和 '日期' 分组，计算平均收益率
    return_grouped = final_data.groupby(['聚类标签', '日期'])['收益率变化'].mean().reset_index()
    
    # 为每个聚类标签绘制折线
    i = 0
    plt.figure(figsize=(12, 12))
    for cluster in df_grouped['聚类标签'].unique():
        if i == 0:
            x = 'red'
        elif i == 1:
            x = 'blue'
        elif i == 2:
            x = 'green'
        df_cluster = return_grouped[return_grouped['聚类标签'] == cluster]
        plt.plot(df_cluster['日期'], df_cluster['收益率变化'], label=f'收益率变化 - 聚类 {cluster}', color=x)
        i += 1
    
    plt.xlabel('日期')
    plt.ylabel('平均收益率')
    plt.title('收益率趋势图')
    
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator(bymonthday=15, interval=3))  # 每三个月显示
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  # 格式化为年-月
    
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    
    # 显示图表
    # plt.savefig(f'个股分析/{market_address}/图片/收益率趋势图', dpi=300)
    plt.show()

## 三维展示图，分别展示簇是0，1，2的代表图

In [14]:
### 聚类0示例 ###
if MODEL != 'Deep-learning/Ours':
    # 复制数据集，确保不会修改原始数据
    merged_data_cleaned = final_data.copy()
    merged_data_cleaned['日期'] = pd.to_datetime(merged_data_cleaned['日期'])
    
    # 选择聚类标签为 0 的数据，找到数据最多的股票编号
    cluster_0_data = merged_data_cleaned[merged_data_cleaned['聚类标签'] == 0]
    top_stock_code = cluster_0_data['股票编号'].value_counts().idxmax()
    top_stock_data = cluster_0_data[cluster_0_data['股票编号'] == top_stock_code]
    
    # 创建 3D 图
    fig = plt.figure(figsize=(50, 50))
    ax = fig.add_subplot(111, projection='3d')
    
    # 提取绘图数据
    x = top_stock_data[EMO_NAME]  # 情绪值作为 x 轴
    y = mdates.date2num(top_stock_data['日期'])  # 时间转换为数值类型
    z = top_stock_data['收益率变化']  # 收益率变化作为 z 轴
    
    # 绘制三维折线图
    ax.plot(x, y, z, label=f"股票 {top_stock_code}", marker='o', linestyle='-', alpha=0.7)
    
    # 设置轴标签
    ax.set_xlabel(EMO_NAME)
    ax.set_ylabel('时间')
    ax.set_zlabel('收益率变化')
    ax.set_title(f'股票 {top_stock_code}: {EMO_NAME} 与收益率变化的三维关系')
    
    # 设置 y 轴时间刻度（每半年一个刻度）
    ax.yaxis.set_major_locator(mdates.MonthLocator(bymonthday=15, interval=6))  # 每半年一个刻度
    ax.yaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  # 按年月显示
    
    # 旋转日期标签防止重叠
    fig.autofmt_xdate()
    
    # 调整布局并保存图像
    plt.legend()
    # plt.savefig(f'个股分析/{market_address}/图片/聚类0_{top_stock_code}.png', dpi=300, bbox_inches='tight')
    plt.show()


In [15]:
### 聚类1示例 ###
if MODEL != 'Deep-learning/Ours':
    # 复制数据集，确保不会修改原始数据
    merged_data_cleaned = final_data.copy()
    merged_data_cleaned['日期'] = pd.to_datetime(merged_data_cleaned['日期'])
    
    # 选择聚类标签为 0 的数据，找到数据最多的股票编号
    cluster_0_data = merged_data_cleaned[merged_data_cleaned['聚类标签'] == 1]
    top_stock_code = cluster_0_data['股票编号'].value_counts().idxmax()
    top_stock_data = cluster_0_data[cluster_0_data['股票编号'] == top_stock_code]
    
    
    # 创建 3D 图
    fig = plt.figure(figsize=(50, 50))
    ax = fig.add_subplot(111, projection='3d')
    
    # 提取绘图数据
    x = top_stock_data[EMO_NAME]  # 情绪值作为 x 轴
    y = mdates.date2num(top_stock_data['日期'])  # 时间转换为数值类型
    z = top_stock_data['收益率变化']  # 收益率变化作为 z 轴
    
    # 绘制三维折线图
    ax.plot(x, y, z, label=f"股票 {top_stock_code}", marker='o', linestyle='-', alpha=0.7)
    
    # 设置轴标签
    ax.set_xlabel(EMO_NAME)
    ax.set_ylabel('时间')
    ax.set_zlabel('收益率变化')
    ax.set_title(f'股票 {top_stock_code}: {EMO_NAME} 与收益率变化的三维关系')
    
    # 设置 y 轴时间刻度（每半年一个刻度）
    ax.yaxis.set_major_locator(mdates.MonthLocator(bymonthday=15, interval=6))  # 每半年一个刻度
    ax.yaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  # 按年月显示
    
    # 旋转日期标签防止重叠
    fig.autofmt_xdate()
    
    # 调整布局并保存图像
    plt.legend()
    # plt.savefig(f'个股分析/{market_address}/图片/聚类1_{top_stock_code}.png', dpi=300, bbox_inches='tight')
    plt.show()


In [16]:
### 聚类2示例 ###
if MODEL != 'Deep-learning/Ours':
    # 复制数据集，确保不会修改原始数据
    merged_data_cleaned = final_data.copy()
    merged_data_cleaned['日期'] = pd.to_datetime(merged_data_cleaned['日期'])
    
    # 选择聚类标签为 0 的数据，找到数据最多的股票编号
    cluster_0_data = merged_data_cleaned[merged_data_cleaned['聚类标签'] == 2]
    top_stock_code = cluster_0_data['股票编号'].value_counts().idxmax()
    top_stock_data = cluster_0_data[cluster_0_data['股票编号'] == top_stock_code]
    
    
    # 创建 3D 图
    fig = plt.figure(figsize=(50, 50))
    ax = fig.add_subplot(111, projection='3d')
    
    # 提取绘图数据
    x = top_stock_data[EMO_NAME]  # 情绪值作为 x 轴
    y = mdates.date2num(top_stock_data['日期'])  # 时间转换为数值类型
    z = top_stock_data['收益率变化']  # 收益率变化作为 z 轴
    
    # 绘制三维折线图
    ax.plot(x, y, z, label=f"股票 {top_stock_code}", marker='o', linestyle='-', alpha=0.7)
    
    # 设置轴标签
    ax.set_xlabel(EMO_NAME)
    ax.set_ylabel('时间')
    ax.set_zlabel('收益率变化')
    ax.set_title(f'股票 {top_stock_code}: {EMO_NAME} 与收益率变化的三维关系')
    
    # 设置 y 轴时间刻度（每半年一个刻度）
    ax.yaxis.set_major_locator(mdates.MonthLocator(bymonthday=15, interval=6))  # 每半年一个刻度
    ax.yaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  # 按年月显示
    
    # 旋转日期标签防止重叠
    fig.autofmt_xdate()
    
    # 调整布局并保存图像
    plt.legend()
    # plt.savefig(f'个股分析/{market_address}/图片/聚类2_{top_stock_code}.png', dpi=300, bbox_inches='tight')
    plt.show()


## 进行聚类间实证分析

#### 读取个股金融数据

In [17]:
# 读取原始数据
microfinancial_data = pd.read_csv(f'{Financial_Data_PATH}/个股微观金融数据.csv', encoding='utf-8')

# 选择重要的列，日期和股票代码一定要包含
columns_to_select = [
    '日期_Date',  # 日期
    '股票代码_Stkcd',  # 股票代码
    '日振幅(%)_Dampltd',  # 日振幅
    '日收益率_Dret',  # 日收益率
    '市盈率_PE',  # 市盈率
    '成交量_Trdvol',  # 成交量
    '流通股日换手率(%)_DTrdTurnR',
    '总市值加权平均日资本收益_Daretmc',
]

# 提取相关列
microfinancial_data = microfinancial_data[columns_to_select]
microfinancial_data['股票代码_Stkcd'] = microfinancial_data['股票代码_Stkcd'].apply(lambda x: str(x).zfill(6))

# microfinancial_data = microfinancial_data[microfinancial_data['日期_Date'] == '2022-03-01'] #用于检验某一天数据
microfinancial_data

,日期_Date,股票代码_Stkcd,日振幅(%)_Dampltd,日收益率_Dret,市盈率_PE,成交量_Trdvol,流通股日换手率(%)_DTrdTurnR,总市值加权平均日资本收益_Daretmc
0,2023-02-01,000100,1.6667,0.0071,276.47,147751077.0,1.0357,0.0145
1,2023-02-02,000100,2.1277,-0.0095,273.86,188929965.0,1.3244,-0.0005
2,2023-02-03,000100,2.1480,-0.0048,272.55,200087013.0,1.4026,-0.0042
3,2023-02-06,000100,1.6787,-0.0072,270.59,134158688.0,0.9404,-0.0084
4,2023-02-07,000100,1.4493,0.0024,271.24,87200030.0,0.6113,0.0036
...,...,...,...,...,...,...,...,...
162617,2024-11-14,601566,3.0340,-0.0255,16.94,3223500.0,0.5610,-0.0146
162618,2024-11-15,601566,2.8643,0.0025,16.98,2824500.0,0.4915,-0.0112
162619,2024-11-18,601566,3.8509,-0.0025,16.94,3729993.0,0.6491,0.0009
162620,2024-11-19,601566,2.3661,0.0137,17.17,3367065.0,0.5859,0.0031


#### 加载之前的情绪聚类数据

In [18]:
final_data

,股票编号,日期,高维情绪变量,交易量,收益率变化,聚类标签,公司名称
0,000002,2021-06-01,"[-0.178, -0.141, 0.283, 0.184, -0.002, 0.299, ...",60990961.0,-0.003745,2.0,万科A
1,000002,2021-06-08,"[-0.276, -0.035, 0.247, 0.326, 0.018, 0.244, -...",44676494.0,0.004626,2.0,万科A
2,000002,2021-06-10,"[-0.083, 0.072, 0.273, 0.26, -0.087, 0.276, -0...",53800776.0,-0.010035,2.0,万科A
3,000002,2021-06-11,"[-0.114, 0.163, 0.381, 0.325, -0.091, 0.088, -...",75853738.0,-0.014035,2.0,万科A
4,000002,2021-06-15,"[-0.169, 0.147, 0.442, 0.245, -0.009, 0.368, -...",89915501.0,-0.020957,2.0,万科A
...,...,...,...,...,...,...,...
9679,688981,2024-11-11,"[-0.055, 0.026, 0.277, 0.147, -0.029, 0.276, -...",138069018.0,0.046660,2.0,中芯国际
9680,688981,2024-11-12,"[-0.134, -0.011, 0.308, 0.208, 0.049, 0.263, -...",108866759.0,-0.036864,2.0,中芯国际
9681,688981,2024-11-13,"[-0.228, 0.028, 0.358, 0.294, -0.137, 0.178, -...",80759477.0,-0.019484,2.0,中芯国际
9682,688981,2024-11-14,"[-0.18, 0.054, 0.392, 0.292, -0.048, 0.215, -0...",76194102.0,-0.022897,2.0,中芯国际


#### 读取市场换手率与市盈率并进行插值

In [19]:
turnover_PE_rate = pd.read_csv(f'{Financial_Data_PATH}/市场整体换手率与市盈率.csv', encoding='utf-8')
turnover_PE_rate['统计日期'] = pd.to_datetime(turnover_PE_rate['统计日期'])

# 按照 '统计日期' 排序
turnover_PE_rate = turnover_PE_rate.sort_values(by='统计日期')

# 对 '平均市盈率' 和 '换手率(总股本)' 进行线性插值
turnover_PE_rate[['平均市盈率', '换手率(总股本)']] = turnover_PE_rate[['平均市盈率', '换手率(总股本)']].interpolate(method='linear', axis=0)

turnover_PE_rate = turnover_PE_rate.drop(columns=['统计口径编码'])
turnover_PE_rate.columns = ['统计日期', '市场平均市盈率', '市场换手率(总股本)']
turnover_PE_rate

,统计日期,市场平均市盈率,市场换手率(总股本)
0,2021-01-04,16.918000,1.140600
12,2021-01-04,25.986305,1.237217
11,2021-01-04,35.054611,1.333834
10,2021-01-04,44.122916,1.430451
8,2021-01-04,53.191221,1.527068
...,...,...,...
12364,2024-12-06,39.185061,2.176139
12363,2024-12-06,14.390000,1.292500
12374,2024-12-06,14.390000,1.292500
12368,2024-12-06,14.390000,1.292500


#### 数据拼接

In [20]:
# 将日期列转换为 datetime 格式
final_data['日期'] = pd.to_datetime(final_data['日期'])
microfinancial_data['日期_Date'] = pd.to_datetime(microfinancial_data['日期_Date'], errors='coerce')

# 使用 merge 进行内连接
result = pd.merge(
    final_data, 
    microfinancial_data, 
    left_on=['日期', '股票编号'],  # 左表的 key
    right_on=['日期_Date', '股票代码_Stkcd'], # 右表的 key
    how='inner'                   # 内连接
)

result = pd.merge(
    result, 
    turnover_PE_rate, 
    left_on=['日期'],  # 左表的 key
    right_on=['统计日期'], # 右表的 key
    how='inner'                   # 内连接
)

# 删除重复的列
result = result.drop(columns=['日期_Date', '股票代码_Stkcd', '交易量', '日收益率_Dret', '统计日期'])

# 修改列的位置，更加美观可视
columns_to_move = ['聚类标签', '公司名称']
result = pd.concat([result.drop(columns=columns_to_move), result[columns_to_move]], axis=1)

# 将成交量除以10000，并以万计数
result['成交量(百万)_Trdvol'] = result['成交量_Trdvol'] / 1_000_000
result = result.drop(columns=['成交量_Trdvol'])

cols_to_check = ['收益率变化', '日振幅(%)_Dampltd', '市盈率_PE', 
                 '成交量(百万)_Trdvol', '流通股日换手率(%)_DTrdTurnR']

# 删除含有空值的行
result = result.dropna(subset=cols_to_check)
result

,股票编号,日期,高维情绪变量,收益率变化,日振幅(%)_Dampltd,市盈率_PE,流通股日换手率(%)_DTrdTurnR,总市值加权平均日资本收益_Daretmc,市场平均市盈率,市场换手率(总股本),聚类标签,公司名称,成交量(百万)_Trdvol
0,000002,2021-06-01,"[-0.178, -0.141, 0.283, 0.184, -0.002, 0.299, ...",-0.003745,1.8727,7.44,0.6272,0.0039,24.523860,1.030223,2.0,万科A,60.990961
1,000002,2021-06-01,"[-0.178, -0.141, 0.283, 0.184, -0.002, 0.299, ...",-0.003745,1.8727,7.44,0.6272,0.0039,31.761720,1.132746,2.0,万科A,60.990961
2,000002,2021-06-01,"[-0.178, -0.141, 0.283, 0.184, -0.002, 0.299, ...",-0.003745,1.8727,7.44,0.6272,0.0039,38.999580,1.235269,2.0,万科A,60.990961
3,000002,2021-06-01,"[-0.178, -0.141, 0.283, 0.184, -0.002, 0.299, ...",-0.003745,1.8727,7.44,0.6272,0.0039,46.237440,1.337792,2.0,万科A,60.990961
4,000002,2021-06-01,"[-0.178, -0.141, 0.283, 0.184, -0.002, 0.299, ...",-0.003745,1.8727,7.44,0.6272,0.0039,53.475301,1.440316,2.0,万科A,60.990961
...,...,...,...,...,...,...,...,...,...,...,...,...,...
116592,603288,2024-11-11,"[0.17, 0.06, 0.068, 0.234, 0.072, 0.144, -0.65...",-0.028694,3.8532,45.78,0.4017,-0.0010,70.557753,3.703170,0.0,海天味业,22.335171
116593,603288,2024-11-11,"[0.17, 0.06, 0.068, 0.234, 0.072, 0.144, -0.65...",-0.028694,3.8532,45.78,0.4017,-0.0010,51.935168,3.099546,0.0,海天味业,22.335171
116594,603288,2024-11-11,"[0.17, 0.06, 0.068, 0.234, 0.072, 0.144, -0.65...",-0.028694,3.8532,45.78,0.4017,-0.0010,33.312584,2.495923,0.0,海天味业,22.335171
116595,603288,2024-11-11,"[0.17, 0.06, 0.068, 0.234, 0.072, 0.144, -0.65...",-0.028694,3.8532,45.78,0.4017,-0.0010,14.690000,1.892300,0.0,海天味业,22.335171


In [21]:
# 查看匹配的公司数
unique_stock_count = result['股票编号'].nunique()
print(unique_stock_count)

19


#### 回归检验

In [22]:
## 单因子：总市值加权平均日资本收益回归检验
df = result.copy()

# 自变量是总市值加权平均日资本收益
X = df[['总市值加权平均日资本收益_Daretmc']]

# 要拟合的因变量列表
y_vars = [
    '收益率变化', '日振幅(%)_Dampltd', '市盈率_PE', 
    '成交量(百万)_Trdvol','流通股日换手率(%)_DTrdTurnR'
]


# 添加常数项（截距项）
X_with_const = sm.add_constant(X)

# 创建一个空的列表存储回归结果
regression_results = []

# 对每个因变量进行回归分析
for y_var in y_vars:
    y = df[y_var]  # 当前的因变量

    # 对股票编号分组进行回归
    grouped = df.groupby('股票编号')
    
    r_squared_list = []
    
    for stock_code, group in grouped:
        X_group = group[['总市值加权平均日资本收益_Daretmc']]
        X_with_const_group = sm.add_constant(X_group)
        y_group = group[y_var]
        
        try:
            # 拟合回归模型
            model = sm.OLS(y_group, X_with_const_group)
            results = model.fit()

            # 提取回归R方
            r_squared_list.append(results.rsquared)
        except Exception as e:
            # 如果回归失败（如R方计算为-inf），跳过
            print(f"回归失败：股票编号 {stock_code}, 错误信息: {e}")
            r_squared_list.append(float('nan'))  # 将其R方设为NaN
    
    # 计算R方的平均值，忽略NaN值
    avg_r_squared = pd.Series(r_squared_list).mean()
    
    # 对于每个因变量，存储回归系数等信息
    model = sm.OLS(y, X_with_const)
    results = model.fit()

    regression_results.append({
        'Variable': y_var,
        'S': round(results.bse['总市值加权平均日资本收益_Daretmc'], 3),  # 标准误差
        'Econ': round(results.params['总市值加权平均日资本收益_Daretmc'], 3),  # 回归系数
        'Constant': round(results.params['const'], 3),  # 截距项，保留三位小数
        'R_squared': round(results.rsquared * 100, 3), # 总体的回归R方，保留三位小数
        'Econ_t_value': round(results.tvalues['总市值加权平均日资本收益_Daretmc'], 3),  # Econ的t统计量，保留三位小数
        'Constant_t_value': round(results.tvalues['const'], 3)  # Constant的t统计量，保留三位小数
    })

# 转换为 DataFrame
regression_df = pd.DataFrame(regression_results)
# regression_df.to_csv(f'个股分析/{market_address}/双因子回归结果/单因子：总市值加权平均日资本收益.csv')
regression_df


,Variable,S,Econ,Constant,R_squared,Econ_t_value,Constant_t_value
0,收益率变化,0.006,1.091,0.001,22.112,181.938,11.641
1,日振幅(%)_Dampltd,0.533,8.629,3.438,0.224,16.184,549.441
2,市盈率_PE,57.149,-123.203,59.781,0.004,-2.156,89.131
3,成交量(百万)_Trdvol,21.662,508.151,64.412,0.470,23.458,253.365
4,流通股日换手率(%)_DTrdTurnR,0.451,11.012,1.498,0.508,24.404,282.825


In [23]:
## 单因子：情绪回归检验
if MODEL != 'Deep-learning/Ours':
    df = result.copy()
    
    # 自变量是情绪值
    X = df[[EMO_NAME]]
    
    # 要拟合的因变量列表
    y_vars = [
        '收益率变化', '日振幅(%)_Dampltd', '市盈率_PE', 
        '成交量(百万)_Trdvol','流通股日换手率(%)_DTrdTurnR'
    ]
    
    # 添加常数项（截距项）
    X_with_const = sm.add_constant(X)
    
    # 创建一个空的列表存储回归结果
    regression_results = []
    
    # 对每个因变量进行回归分析
    for y_var in y_vars:
        y = df[y_var]  # 当前的因变量
    
        # 对股票编号分组进行回归
        grouped = df.groupby('股票编号')
        
        for stock_code, group in grouped:
            X_group = group[[EMO_NAME]]
            X_with_const_group = sm.add_constant(X_group)
            y_group = group[y_var]
            
            # 拟合回归模型
            model = sm.OLS(y_group, X_with_const_group)
            results = model.fit()
    
            # 提取回归R方
            r_squared_list.append(results.rsquared)
        
        # 计算R方的平均值
        avg_r_squared = sum(r_squared_list) / len(r_squared_list)
        
        # 对于每个因变量，存储回归系数等信息
        model = sm.OLS(y, X_with_const)
        results = model.fit()
    
        regression_results.append({
            'Variable': y_var,
            'S': round(results.bse[EMO_NAME], 3),  # 标准误差，保留三位小数
            'Econ': round(results.params[EMO_NAME], 3),  # 回归系数，保留三位小数
            'Constant': round(results.params['const'], 3),  # 截距项，保留三位小数
            'R_squared': round(results.rsquared * 100, 3), # 总体的回归R方，保留三位小数
            'Econ_t_value': round(results.tvalues[EMO_NAME], 3),  # Econ的t统计量，保留三位小数
            'Constant_t_value': round(results.tvalues['const'], 3)  # Constant的t统计量，保留三位小数
        })
else:
    df = result.copy()
    
    # 拆分高维情绪变量为多个列（多变量）
    emo_array = np.array(df['高维情绪变量'].tolist())  # shape: (样本数, 维度)
    emo_df = pd.DataFrame(emo_array, columns=[f'emotion_{i}' for i in range(emo_array.shape[1])])
    
    # 合并回df
    df = pd.concat([df.reset_index(drop=True), emo_df], axis=1)
    
    # 构造多维自变量X
    EMO_COLUMNS = [f'emotion_{i}' for i in range(emo_array.shape[1])]
    X = df[EMO_COLUMNS]
    
    # 要拟合的因变量列表
    y_vars = [
        '收益率变化', '日振幅(%)_Dampltd', '市盈率_PE', 
        '成交量(百万)_Trdvol','流通股日换手率(%)_DTrdTurnR'
    ]
    
    # 添加常数项（截距项）
    X_with_const = sm.add_constant(X)
    
    regression_results = []

    for y_var in y_vars:
        y = df[y_var]
        
        # 每个股票编号做回归，统计 R²
        grouped = df.groupby('股票编号')
        
        # 全部数据回归
        model = sm.OLS(y, X_with_const)
        results = model.fit()

        # 提取部分指标（示例提取第一个变量的参数）
        econ_param = results.params[EMO_COLUMNS[0]]
        econ_bse = results.bse[EMO_COLUMNS[0]]
        econ_t = results.tvalues[EMO_COLUMNS[0]]
        
        regression_results.append({
            'Variable': y_var,
            'S': round(econ_bse, 3),
            'Econ': round(econ_param, 3),
            'Constant': round(results.params['const'], 3),
            'R_squared_global': round(results.rsquared * 100, 3),
            'Econ_t_value': round(econ_t, 3),
            'Constant_t_value': round(results.tvalues['const'], 3)
        })
    


# 转换为 DataFrame
regression_df = pd.DataFrame(regression_results)
# regression_df.to_csv(f'个股分析/{market_address}/双因子回归结果/单因子：{EMO_NAME}.csv')
regression_df


,Variable,S,Econ,Constant,R_squared_global,Econ_t_value,Constant_t_value
0,收益率变化,0.010,-0.023,-0.103,14.715,-2.400,-2.582
1,日振幅(%)_Dampltd,0.731,-1.680,17.066,20.170,-2.296,5.626
2,市盈率_PE,79.483,-232.155,-188.685,17.770,-2.921,-0.572
3,成交量(百万)_Trdvol,27.579,-13.941,103.630,31.414,-0.505,0.906
4,流通股日换手率(%)_DTrdTurnR,0.611,-2.051,9.494,22.321,-3.354,3.744


In [24]:
## 总体双因子回归检验
if MODEL != 'Deep-learning/Ours':
    df = result.copy()
    
    # 自变量是情绪值和总市值加权平均日资本收益
    X = df[[EMO_NAME, '总市值加权平均日资本收益_Daretmc']]
    
    # 要拟合的因变量列表
    y_vars = [
        '收益率变化', '日振幅(%)_Dampltd', '市盈率_PE', 
        '成交量(百万)_Trdvol','流通股日换手率(%)_DTrdTurnR'
    ]
    
    # 添加常数项（截距项）
    X_with_const = sm.add_constant(X)
    
    # 创建一个空的列表存储回归结果
    regression_results = []
    
    # 对每个因变量进行回归分析
    for y_var in y_vars:
        y = df[y_var]  # 当前的因变量
    
        # 对股票编号分组进行回归
        grouped = df.groupby('股票编号')
   
        # 对于每个因变量，存储回归系数等信息
        model = sm.OLS(y, X_with_const)
        results = model.fit()
    
        regression_results.append({
            'Variable': y_var,
            'S': round(results.bse[EMO_NAME], 3),  # 标准误差，保留三位小数
            'Econ': round(results.params[EMO_NAME], 3),  # 回归系数，保留三位小数
            'Constant': round(results.params['const'], 3),  # 截距项，保留三位小数
            'R_squared': round(results.rsquared * 100, 3), # 总体的回归R方，保留三位小数
            'Econ_t_value': round(results.tvalues[EMO_NAME], 3),  # Econ的t统计量，保留三位小数
            'Constant_t_value': round(results.tvalues['const'], 3)  # Constant的t统计量，保留三位小数
        })

else:
    df = result.copy()
    
    # 拆分高维情绪变量为多个列（多变量）
    emo_array = np.array(df['高维情绪变量'].tolist())  # shape: (样本数, 维度)
    emo_df = pd.DataFrame(emo_array, columns=[f'emotion_{i}' for i in range(emo_array.shape[1])])
    
    # 合并回df
    df = pd.concat([df.reset_index(drop=True), emo_df], axis=1)
    
    # 构造多维自变量X
    EMO_COLUMNS = [f'emotion_{i}' for i in range(emo_array.shape[1])]
    X = df[['总市值加权平均日资本收益_Daretmc'] + EMO_COLUMNS]
    
    # 要拟合的因变量列表
    y_vars = [
        '收益率变化', '日振幅(%)_Dampltd', '市盈率_PE', 
        '成交量(百万)_Trdvol','流通股日换手率(%)_DTrdTurnR'
    ]
    
    # 添加常数项（截距项）
    X_with_const = sm.add_constant(X)
    
    regression_results = []

    for y_var in y_vars:
        y = df[y_var]
        
        # 每个股票编号做回归，统计 R²
        grouped = df.groupby('股票编号')

        # 全部数据回归
        model = sm.OLS(y, X_with_const)
        results = model.fit()

        # 提取部分指标（示例提取第一个变量的参数）
        econ_param = results.params[EMO_COLUMNS[0]]
        econ_bse = results.bse[EMO_COLUMNS[0]]
        econ_t = results.tvalues[EMO_COLUMNS[0]]
        
        regression_results.append({
            'Variable': y_var,
            'S': round(econ_bse, 3),
            'Econ': round(econ_param, 3),
            'Constant': round(results.params['const'], 3),
            'R_squared': round(results.rsquared * 100, 3),
            'Econ_t_value': round(econ_t, 3),
            'Constant_t_value': round(results.tvalues['const'], 3)
        })
    

# 转换为 DataFrame
regression_df = pd.DataFrame(regression_results)
# regression_df.to_csv(f'个股分析/{market_address}/双因子回归结果/双因子：总体分析.csv')
regression_df


,Variable,S,Econ,Constant,R_squared,Econ_t_value,Constant_t_value
0,收益率变化,0.009,-0.017,-0.058,31.799,-1.995,-1.617
1,日振幅(%)_Dampltd,0.731,-1.635,17.405,20.325,-2.238,5.743
2,市盈率_PE,79.480,-233.300,-197.443,17.779,-2.935,-0.599
3,成交量(百万)_Trdvol,27.503,-11.113,125.258,31.795,-0.404,1.098
4,流通股日换手率(%)_DTrdTurnR,0.610,-1.989,9.969,22.743,-3.262,3.942


In [25]:
## 聚类0的回归检验
if MODEL != 'Deep-learning/Ours':
    df = result[result['聚类标签'] == 0].copy()
    
    # 自变量是情绪值和总市值加权平均日资本收益
    X = df[[EMO_NAME, '总市值加权平均日资本收益_Daretmc']]
    
    # 要拟合的因变量列表
    y_vars = [
        '收益率变化', '日振幅(%)_Dampltd', '市盈率_PE', 
        '成交量(百万)_Trdvol','流通股日换手率(%)_DTrdTurnR'
    ]
    
    
    # 添加常数项（截距项）
    X_with_const = sm.add_constant(X)
    
    # 创建一个空的列表存储回归结果
    regression_results = []
    
    # 对每个因变量进行回归分析
    for y_var in y_vars:
        y = df[y_var]  # 当前的因变量
    
        # 对股票编号分组进行回归
        grouped = df.groupby('股票编号')
        
        # 对于每个因变量，存储回归系数等信息
        model = sm.OLS(y, X_with_const)
        results = model.fit()
    
        regression_results.append({
            'Variable': y_var,
            'S': round(results.bse[EMO_NAME], 3),  # 标准误差，保留三位小数
            'Econ': round(results.params[EMO_NAME], 3),  # 回归系数，保留三位小数
            'Constant': round(results.params['const'], 3),  # 截距项，保留三位小数
            'R_squared': round(results.rsquared * 100, 3), # 总体的回归R方，保留三位小数
            'Econ_t_value': round(results.tvalues[EMO_NAME], 3),  # Econ的t统计量，保留三位小数
            'Constant_t_value': round(results.tvalues['const'], 3)  # Constant的t统计量，保留三位小数
        })

else:
    df = result[result['聚类标签'] == 0].copy()

    # 拆分高维情绪变量为多个列（多变量）
    emo_array = np.array(df['高维情绪变量'].tolist())  # shape: (样本数, 维度)
    emo_df = pd.DataFrame(emo_array, columns=[f'emotion_{i}' for i in range(emo_array.shape[1])])
    
    # 合并回df
    df = pd.concat([df.reset_index(drop=True), emo_df], axis=1)
    
    # 构造多维自变量X
    EMO_COLUMNS = [f'emotion_{i}' for i in range(emo_array.shape[1])]
    X = df[['总市值加权平均日资本收益_Daretmc'] + EMO_COLUMNS]
    
    # 要拟合的因变量列表
    y_vars = [
        '收益率变化', '日振幅(%)_Dampltd', '市盈率_PE', 
        '成交量(百万)_Trdvol','流通股日换手率(%)_DTrdTurnR'
    ]
    
    # 添加常数项（截距项）
    X_with_const = sm.add_constant(X)
    
    regression_results = []

    for y_var in y_vars:
        y = df[y_var]
        
        # 每个股票编号做回归，统计 R²
        grouped = df.groupby('股票编号')
        
        # 全部数据回归
        model = sm.OLS(y, X_with_const)
        results = model.fit()

        # 提取部分指标（示例提取第一个变量的参数）
        econ_param = results.params[EMO_COLUMNS[0]]
        econ_bse = results.bse[EMO_COLUMNS[0]]
        econ_t = results.tvalues[EMO_COLUMNS[0]]
        
        regression_results.append({
            'Variable': y_var,
            'S': round(econ_bse, 3),
            'Econ': round(econ_param, 3),
            'Constant': round(results.params['const'], 3),
            'R_squared': round(results.rsquared * 100, 3),
            'Econ_t_value': round(econ_t, 3),
            'Constant_t_value': round(results.tvalues['const'], 3)
        })
    
# 转换为 DataFrame
regression_df = pd.DataFrame(regression_results)
# regression_df.to_csv(f'个股分析/{market_address}/双因子回归结果/双因子：聚类0检验.csv')
regression_df


,Variable,S,Econ,Constant,R_squared,Econ_t_value,Constant_t_value
0,收益率变化,0.013,-0.009,0.110,37.491,-0.679,1.842
1,日振幅(%)_Dampltd,1.027,-3.619,46.938,31.702,-3.523,9.743
2,市盈率_PE,115.966,-22.233,-2476.121,26.097,-0.192,-4.553
3,成交量(百万)_Trdvol,44.024,-27.448,-234.740,48.501,-0.623,-1.137
4,流通股日换手率(%)_DTrdTurnR,0.914,-2.984,15.228,29.131,-3.264,3.552


In [26]:
## 聚类1的回归检验
if MODEL != 'Deep-learning/Ours':
    df = result[result['聚类标签'] == 1].copy()
    
    # 自变量是情绪值和总市值加权平均日资本收益
    X = df[[EMO_NAME, '总市值加权平均日资本收益_Daretmc']]
    
    # 要拟合的因变量列表
    y_vars = [
        '收益率变化', '日振幅(%)_Dampltd', '市盈率_PE', 
        '成交量(百万)_Trdvol','流通股日换手率(%)_DTrdTurnR'
    ]
    
    
    # 添加常数项（截距项）
    X_with_const = sm.add_constant(X)
    
    # 创建一个空的列表存储回归结果
    regression_results = []
    
    # 对每个因变量进行回归分析
    for y_var in y_vars:
        y = df[y_var]  # 当前的因变量
    
        # 对股票编号分组进行回归
        grouped = df.groupby('股票编号')

        # 对于每个因变量，存储回归系数等信息
        model = sm.OLS(y, X_with_const)
        results = model.fit()
    
        regression_results.append({
            'Variable': y_var,
            'S': round(results.bse[EMO_NAME], 3),  # 标准误差，保留三位小数
            'Econ': round(results.params[EMO_NAME], 3),  # 回归系数，保留三位小数
            'Constant': round(results.params['const'], 3),  # 截距项，保留三位小数
            'R_squared': round(results.rsquared * 100, 3), # 总体的回归R方，保留三位小数
            'Econ_t_value': round(results.tvalues[EMO_NAME], 3),  # Econ的t统计量，保留三位小数
            'Constant_t_value': round(results.tvalues['const'], 3)  # Constant的t统计量，保留三位小数
        })
else:
    df = result[result['聚类标签'] == 1].copy()
    
    # 拆分高维情绪变量为多个列（多变量）
    emo_array = np.array(df['高维情绪变量'].tolist())  # shape: (样本数, 维度)
    print('(样本数, 维度):', emo_array.shape)
    emo_df = pd.DataFrame(emo_array, columns=[f'emotion_{i}' for i in range(emo_array.shape[1])])
    
    # 合并回df
    df = pd.concat([df.reset_index(drop=True), emo_df], axis=1)
    
    # 构造多维自变量X
    EMO_COLUMNS = [f'emotion_{i}' for i in range(emo_array.shape[1])]
    X = df[['总市值加权平均日资本收益_Daretmc'] + EMO_COLUMNS]
    
    # 要拟合的因变量列表
    y_vars = [
        '收益率变化', '日振幅(%)_Dampltd', '市盈率_PE', 
        '成交量(百万)_Trdvol','流通股日换手率(%)_DTrdTurnR'
    ]
    
    # 添加常数项（截距项）
    X_with_const = sm.add_constant(X)
    
    regression_results = []

    for y_var in y_vars:
        y = df[y_var]
        
        # 每个股票编号做回归，统计 R²
        grouped = df.groupby('股票编号')    

        # 全部数据回归
        model = sm.OLS(y, X_with_const)
        results = model.fit()

        # 提取部分指标（示例提取第一个变量的参数）
        econ_param = results.params[EMO_COLUMNS[0]]
        econ_bse = results.bse[EMO_COLUMNS[0]]
        econ_t = results.tvalues[EMO_COLUMNS[0]]
        
        regression_results.append({
            'Variable': y_var,
            'S': round(econ_bse, 3),
            'Econ': round(econ_param, 3),
            'Constant': round(results.params['const'], 3),
            'R_squared': round(results.rsquared * 100, 3),
            'Econ_t_value': round(econ_t, 3),
            'Constant_t_value': round(results.tvalues['const'], 3)
        })

# 转换为 DataFrame
regression_df = pd.DataFrame(regression_results)
# regression_df.to_csv(f'个股分析/{market_address}/双因子回归结果/双因子：聚类1检验.csv')
regression_df


(样本数, 维度): (17927, 768)


,Variable,S,Econ,Constant,R_squared,Econ_t_value,Constant_t_value
0,收益率变化,0.025,-0.015,-0.816,69.209,-0.590,-7.005
1,日振幅(%)_Dampltd,2.286,-3.541,-35.772,61.673,-1.549,-3.348
2,市盈率_PE,345.789,-122.119,-9474.820,64.791,-0.353,-5.863
3,成交量(百万)_Trdvol,16.636,-27.822,-785.271,56.516,-1.672,-10.100
4,流通股日换手率(%)_DTrdTurnR,1.351,2.700,-22.601,63.469,1.998,-3.579


In [27]:
## 聚类2的回归检验
if MODEL != 'Deep-learning/Ours':
    df = result[result['聚类标签'] == 2].copy()
    
    # 自变量是情绪值和总市值加权平均日资本收益
    X = df[[EMO_NAME, '总市值加权平均日资本收益_Daretmc']]
    
    # 要拟合的因变量列表
    y_vars = [
        '收益率变化', '日振幅(%)_Dampltd', '市盈率_PE', 
        '成交量(百万)_Trdvol','流通股日换手率(%)_DTrdTurnR'
    ]
    
    
    # 添加常数项（截距项）
    X_with_const = sm.add_constant(X)
    
    # 创建一个空的列表存储回归结果
    regression_results = []
    
    # 对每个因变量进行回归分析
    for y_var in y_vars:
        y = df[y_var]  # 当前的因变量
    
        # 对股票编号分组进行回归
        grouped = df.groupby('股票编号')
    
        # 对于每个因变量，存储回归系数等信息
        model = sm.OLS(y, X_with_const)
        results = model.fit()
    
        regression_results.append({
            'Variable': y_var,
            'S': round(results.bse[EMO_NAME], 3),  # 标准误差，保留三位小数
            'Econ': round(results.params[EMO_NAME], 3),  # 回归系数，保留三位小数
            'Constant': round(results.params['const'], 3),  # 截距项，保留三位小数
            'R_squared': round(results.rsquared * 100, 3), # 总体的回归R方，保留三位小数
            'Econ_t_value': round(results.tvalues[EMO_NAME], 3),  # Econ的t统计量，保留三位小数
            'Constant_t_value': round(results.tvalues['const'], 3)  # Constant的t统计量，保留三位小数
        })
else:
    df = result[result['聚类标签'] == 2].copy()
    
    # 拆分高维情绪变量为多个列（多变量）
    emo_array = np.array(df['高维情绪变量'].tolist())  # shape: (样本数, 维度)
    print('(样本数, 维度):', emo_array.shape)
    emo_df = pd.DataFrame(emo_array, columns=[f'emotion_{i}' for i in range(emo_array.shape[1])])
    
    # 合并回df
    df = pd.concat([df.reset_index(drop=True), emo_df], axis=1)
    
    # 构造多维自变量X
    EMO_COLUMNS = [f'emotion_{i}' for i in range(emo_array.shape[1])]
    X = df[['总市值加权平均日资本收益_Daretmc'] + EMO_COLUMNS]
    
    # 要拟合的因变量列表
    y_vars = [
        '收益率变化', '日振幅(%)_Dampltd', '市盈率_PE', 
        '成交量(百万)_Trdvol','流通股日换手率(%)_DTrdTurnR'
    ]
    
    # 添加常数项（截距项）
    X_with_const = sm.add_constant(X)
    
    regression_results = []

    for y_var in y_vars:
        y = df[y_var]
        
        # 每个股票编号做回归，统计 R²
        grouped = df.groupby('股票编号')
        r_squared_list = []
        
        # 全部数据回归
        model = sm.OLS(y, X_with_const)
        results = model.fit()

        # 提取部分指标（示例提取第一个变量的参数）
        econ_param = results.params[EMO_COLUMNS[0]]
        econ_bse = results.bse[EMO_COLUMNS[0]]
        econ_t = results.tvalues[EMO_COLUMNS[0]]
        
        regression_results.append({
            'Variable': y_var,
            'S': round(econ_bse, 3),
            'Econ': round(econ_param, 3),
            'Constant': round(results.params['const'], 3),
            'R_squared': round(results.rsquared * 100, 3),
            'Econ_t_value': round(econ_t, 3),
            'Constant_t_value': round(results.tvalues['const'], 3)
        })

# 转换为 DataFrame
regression_df = pd.DataFrame(regression_results)
# regression_df.to_csv(f'个股分析/{market_address}/双因子回归结果/双因子：聚类2检验.csv')
regression_df


(样本数, 维度): (45617, 768)


,Variable,S,Econ,Constant,R_squared,Econ_t_value,Constant_t_value
0,收益率变化,0.014,0.041,-0.401,38.983,2.976,-5.883
1,日振幅(%)_Dampltd,1.133,-3.360,11.003,31.369,-2.966,1.983
2,市盈率_PE,56.653,-273.151,-485.970,41.341,-4.821,-1.752
3,成交量(百万)_Trdvol,43.377,-293.721,-725.347,35.597,-6.771,-3.415
4,流通股日换手率(%)_DTrdTurnR,1.025,-4.997,-0.397,36.590,-4.877,-0.079
